In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy.stats import norm
from math import *

In [2]:
class BlackScholes:
    
        
    def ft_bs_pricing(self, Option_type,S,K,T,r,rf,sigma):
        
        global d1
        global d2
        d1 = (log(S / K) + (r - rf + sigma ** 2 / 2) * T) / (sigma * sqrt(T)) 
        d2 = d1 - sigma* sqrt(T) 
        
        if Option_type == "Call":
            option_price = S* exp(-rf*T)* norm.cdf(d1) - K * exp(-r*T)*norm.cdf(d2)
            return option_price
        elif Option_type == "Put":
            option_price = K* exp(-r*T)* norm.cdf(-d2) - S * exp(-rf*T)*norm.cdf(-d1)
            return option_price
        else:
            return "incorrect parameters"
       
   
    def ft_delta(self, Option_type,S,K,T,r,rf,sigma):
        if Option_type == "Call":
            delta = exp(-rf*T) * norm.cdf(d1)   
        elif Option_type == "Put":
            delta = exp(-rf*T) * (norm.cdf(d1)-1)
        return delta
    
    
    def ft_gamma(self, Option_type,S,K,T,r,rf,sigma):
        return (norm.pdf(d1)*exp(-rf*T) )/ ( S*sigma*sqrt(T) )
    
    def ft_vega(self, Option_type,S,K,T,r,rf,sigma):
        return S*sqrt(T)*norm.pdf(d1)*exp(-rf*T)
    
    def ft_theta(self, Option_type,S,K,T,r,rf,sigma):
        a = (-S *norm.pdf(d1) *sigma* exp(-rf*T)) /(2*sqrt(T))
        b = rf * S * norm.cdf(d1) * exp(-rf*T) 
        c = r * K * exp(-r*T)* norm.cdf(d2)
        b2 = rf * S * norm.cdf(-d1) * exp(-rf*T) 
        c2 = r * K * exp(-r*T)* norm.cdf(-d2)
        if Option_type == "Call":
             theta = a+b-c   
        elif Option_type == "Put":
            theta = a-b2+c2 
        return theta
    
    def ft_rho_dom(self, Option_type,S,K,T,r,rf,sigma):
        if Option_type == "Call":
            rho_dom = K * T *exp(-r*T) * norm.pdf(d2)   
        elif Option_type == "Put":
            rho_dom = -K * T *exp(-r*T) * norm.pdf(-d2)  
        return rho_dom
    
    def ft_rho_fgn(self, Option_type,S,K,T,r,rf,sigma):
        if Option_type == "Call":
            rho_fgn = -S * T *exp(-rf*T) * norm.pdf(d1)   
        elif Option_type == "Put":
            rho_fgn = S * T *exp(-rf*T) * norm.pdf(-d1)  
        return rho_fgn
    
    def ft_vanna(self, Option_type,S,K,T,r,rf,sigma):
        return -exp(-rf*T) * norm.cdf(d1) * d2 / sigma 
    
    
    def __init__(self,Option_type,S,K,T,r,rf,sigma):
        
        self.option_type = Option_type
        self.asset_price = S
        self.strike = K
        self.time_to_exp = T
        self.d_rate = r
        self.f_rate = rf
        self.asset_volatility = sigma
        
        self.price = self.ft_bs_pricing(Option_type,S,K,T,r,rf,sigma)
        
        
        self.delta = self.ft_delta(Option_type,S,K,T,r,rf,sigma)
        self.gamma = self.ft_gamma(Option_type,S,K,T,r,rf,sigma)
        self.theta = self.ft_theta(Option_type,S,K,T,r,rf,sigma)
        self.vega = self.ft_vega(Option_type,S,K,T,r,rf,sigma)
        self.rho_dom = self.ft_rho_dom(Option_type,S,K,T,r,rf,sigma)
        self.rho_fgn = self.ft_rho_fgn(Option_type,S,K,T,r,rf,sigma)
        self.vanna = self.ft_vanna(Option_type,S,K,T,r,rf,sigma)
       

## Risked based method

In [15]:
bs = BlackScholes("Call",1.2034,1.1826, 0.5 ,0.00527843, 0.00137174, 0.0821)

price1 = bs.price
delta1 = bs.delta 
gamma1 = bs.gamma 
vega1 = bs.vega
theta1 = bs.theta
rho_dom1 = bs.rho_dom
rho_fgn1 = bs.rho_fgn

bs2 = BlackScholes("Call",1.2050,1.1826, 7/12 ,0.00527843, 0.00137174, 0.0830)
price2 = bs2.price

var_spot = 0.0016
var_vol = 0.0009
var_time = -1/12
var_rho_dom = 0.00
var_rho_fgn = 0.00

# taylor series approx : f(x) = f(x0) + f'(x0)(x-x0) + f''(x0)(x-x0)**2 +... 
PnL_spot =  delta1*var_spot + 0.5*gamma1*var_spot**2 #+ (1/6)*(bs2.gamma -bs.gamma)*var_spot**2 
PnL_vol = vega1*var_vol
PnL_time = -theta1*var_time
PnL_dom = rho_dom1*var_rho_dom 
PnL_fgn = rho_fgn1*var_rho_fgn
PnL = PnL_spot  + PnL_time + PnL_vol + PnL_dom + PnL_fgn

Real_PnL = price2 - price1
print("Total PnL",PnL)
print("Observed PnL",Real_PnL)
print(end = '\n')

print("Impact of price change", round(PnL_spot/Real_PnL*100,2),"%")
print("Impact of time", round(PnL_time/Real_PnL*100,2),"%")
print("Impact of volatility", round(PnL_vol/Real_PnL*100,2),"%")
print("Impact of domestic rate", round(PnL_dom/Real_PnL*100,2),"%")
print("Impact of foreign rate", round(PnL_fgn/Real_PnL*100,2),"%")
print("Unexplained PnL",round( (Real_PnL - PnL) /Real_PnL*100,2),"%") 


Total PnL -0.0010876194359203304
Observed PnL 0.0036581012365731747

Impact of price change 28.23 %
Impact of time -65.78 %
Impact of volatility 7.81 %
Impact of domestic rate 0.0 %
Impact of foreign rate 0.0 %
Unexplained PnL 129.73 %


In [10]:
# Maturity
import time
from datetime import datetime

T1 = '21/10/2020'
T2 = '22/04/2021'
t = pd.to_datetime(T2) - pd.to_datetime(T1)
t.days/365

0.5013698630136987

In [39]:
vol1 = 8.21
vol2 = 8.23
dom1 = 0.527843
dom2 = 0.50696
fgn1 = 0.137174
fgn2 = 0.138141
fgn2-fgn1

price1 = 19450.354
price2 = 19405.79
print(price2-price1)

-44.56399999999849


In [42]:
delta = -920.511
gamma = -30.939
theta = 54.528
vega = -25.139
rho_dom = 0
rho_fgn = 0



# taylor series approx : f(x) = f(x0) + f'(x0)(x-x0) + f''(x0)(x-x0)**2 +... 
PnL_spot =  delta*var_spot + 0.5*gamma*var_spot**2 #+ (1/6)*(bs2.gamma -bs.gamma)*var_spot**2 
PnL_vol = vega*var_vol
PnL_time = -theta*var_time
PnL_dom = rho_dom*var_rho_dom 
PnL_fgn = rho_fgn*var_rho_fgn
PnL = PnL_spot  + PnL_time + PnL_vol + PnL_dom + PnL_fgn

"""
Real_PnL = price2 - price1
print("Total PnL",PnL)
print("Observed PnL",Real_PnL)
print(end = '\n')

print("Impact of price change", round(PnL_spot/Real_PnL*100,2),"%")
print("Impact of time", round(PnL_time/Real_PnL*100,2),"%")
print("Impact of volatility", round(PnL_vol/Real_PnL*100,2),"%")
print("Impact of domestic rate", round(PnL_dom/Real_PnL*100,2),"%")
print("Impact of foreign rate", round(PnL_fgn/Real_PnL*100,2),"%")
print("Unexplained PnL",round( (Real_PnL - PnL) /Real_PnL*100,2),"%") 
"""

print(PnL_spot)
print(PnL_vol)
print(PnL_time)
print(PnL)

-0.0
-0.0050278
-0.1493917808219178
-0.1544195808219178
